# Jet Vane Unit Design Document
Design calculations for the hover-jet jet vane actuator unit


In [1]:
import numpy as np
import pint
ureg = pint.UnitRegistry()
print('hello')

hello


## Requirements and variables flow
Required moment on vehicle drives jet vane size
TODO

## Assumptions



In [2]:
r_v = 0.2 * ureg.m   # Vane distance from vehicle center of mass


## Required moment on vehicle
### Required moment on vehicle: angular acceleration
TODO

### Required moment on vehicle: center of mass misalignment
TODO

In [3]:
r_T = 5 * ureg.mm    # Thrust / com misalignment
F_T = 160 * ureg.N    # Thrust force

# Required vane force to compensate for com misalignment
F_v = F_T  * r_T / r_v
F_v.ito(ureg.N)
print('Vane force F_v = {:~P}'.format(F_v))

Vane force F_v = 4.0 N


## Jet flow properties
TODO


In [4]:
T_e = 900 * ureg.K    # Exhaust exit temperature
p_e = 100 * ureg.kPa   # Exhaust exit pressure
R_e = 287 * ureg.J / ureg.kg / ureg.K   # Gas constant
rho_e = p_e / (R_e * T_e)
rho_e.ito(ureg.kg * ureg.m**-3)   # Exhaust density
v_e_full_throttle = 442 * ureg.m / ureg.s   # Exhaust gas velocity at full throttle
gamma = 1.25    # Ratio of specific heats
sonic_e = (gamma * R_e * T_e)**0.5
sonic_e.ito(ureg.m / ureg.s)

print('Exhaust gas density rho_e = {:~P.2}'.format(rho_e))
print('Exit Mach number, full throttle = {:~P.2}'.format(v_e_full_throttle / sonic_e))


Exhaust gas density rho_e = 0.39 kg / m ** 3
Exit Mach number, full throttle = 0.78


## Jet vane size
Let's try to achieve the full desired contorl authority at $1/2$ the full-throttle $v_e$ (roughly 1/4 throttle). Keep the jet vane angle of attack $\alpha$ to be 6 degrees at most.

For a wing with thin, symmetric airfoil, the lift coefficient is (including compressiblity and finite span corrections) (see  [vt AOE3104 notes](http://www.dept.aoe.vt.edu/~lutze/AOE3104/airfoilwings.pdf)):

\begin{equation}
C_L = \frac{2 \pi A_R^2}{2 + \sqrt{A_R^2 (1-M^2) +4}} \alpha
\end{equation}

where $A_R$ is the aspect ratio. The required vane force is

\begin{equation}
\frac{F_v}{2} = \frac{1}{2} \rho_e v_e^2 C_L S_v
\end{equation}

where $S_v$ is the vane reference area and the lefthand side is divided by 2 because there are 2 vanes per axis.


In [14]:
# Throttle setting adjustment
throttle = 0.25   # Throttle setting at which to size vanes
# rough approx. - changing throttle would also change exhuast gas temperature and thus density
v_e = v_e_full_throttle * (throttle)**0.5   # Exit velocity at throttle setting
M_e = v_e / sonic_e   # Exit mach number at throttle setting
F_v = F_T * throttle * r_T / r_v  # required vane force due to cm misalign


A_R = 2   # Jet vane aspect ratio
alpha = np.deg2rad(6)

# jet vane lift coeff
C_L = 2 * np.pi * A_R / (2 + (A_R**2 * (1 - M_e**2) + 4)**0.5) * alpha
print('Jet vane lift coeff C_L = {:.2f}'.format(C_L))

# Jet vane area
S_v = F_v / (rho_e * v_e**2 * C_L)
S_v.ito(ureg.m**2)
print('Jet vane area S_v = {:~P.2}'.format(S_v))

# jet vane span
b_v = (A_R * S_v)**0.5
b_v.ito(ureg.m)
print('jet vane span b_v = {:~P.3}'.format(b_v.to(ureg.mm)))

# Jet vane span sanity check
b_v_max = 20 * ureg.mm   # Max possible vane (effective) span, a little less than the engine nozzle exit radius
if b_v > b_v_max:
    print('Vane span is too big!')

Jet vane lift coeff C_L = 0.28 dimensionless
Jet vane area S_v = 0.00019 m ** 2
jet vane span b_v = 19.5 mm
